In [13]:
%pwd

'c:\\Users\\keerthana bhat\\OneDrive\\Desktop\\OpenTurf project\\IT-Helpdesk-Chatbot\\research'

In [14]:
import os
os.chdir("../")

In [15]:
%pwd

'c:\\Users\\keerthana bhat\\OneDrive\\Desktop\\OpenTurf project\\IT-Helpdesk-Chatbot'

In [16]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [18]:
# extract data from pdf
def load_pdf_file(data):
    loader= DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents= loader.load()
    return documents

In [19]:
extracted_data= load_pdf_file(data='data/')

In [6]:
#extracted_data

In [20]:
# split data into chunks
def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [21]:
text_chunks= text_split(extracted_data)
print("Length of text chunks", len(text_chunks))

Length of text chunks 461


In [22]:
# download embedding model from Hugging Face
from langchain.embeddings import HuggingFaceEmbeddings

In [23]:
#download embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings= HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [25]:
embeddings= download_hugging_face_embeddings()

In [ ]:
#query_result= embeddings.embed_query("Hello world")
#print("Length", len(query_result))

Length 384


In [37]:
import os 
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
load_dotenv()

True

In [38]:
PINECONE_API_KEY= os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY= os.environ.get('OPENAI_API_KEY')

In [26]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name= "testbot"

pc.create_index(
    name= index_name,
    dimension= 384,
    metric= "cosine",
    spec= ServerlessSpec(
        cloud="aws",
        region="us-east-1",

    )

)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '468dd16622111989668310260add992c', 'date': 'Fri, 27 Jun 2025 07:44:59 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [39]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [27]:
#embed each chunk and upsert the embeddings into your Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch= PineconeVectorStore.from_documents(
    documents= text_chunks,
    embedding= embeddings,
    index_name= index_name
)

In [28]:
# load existing index
from langchain_pinecone import PineconeVectorStore
# embed each chunk and upsert the embeddings into your Pinecone index
docsearch= PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [29]:
docsearch

In [30]:
retriever= docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [31]:
retriever_docs= retriever.invoke("how to change my password?")

In [32]:
retriever_docs

[Document(id='98ee1273-e4b0-4758-94a8-e2731cafcf35', metadata={'author': 'Mark Edward Soper', 'codemantra, llc': 'http://www.codemantra.com', 'creationdate': '2014-09-18T21:27:50+05:30', 'creator': 'Adobe InDesign CS5 (7.0)', 'moddate': '2014-10-28T12:08:42-04:00', 'page': 78.0, 'page_label': '54', 'producer': 'Acrobat Distiller 8.0.0 (Windows)', 'source': 'data\\sample_kb.pdf', 'title': 'The PC and Gadget Help Desk: A Do-It-Yourself Guide To', 'total_pages': 102.0, 'universal pdf': 'The process that creates this PDF constitutes a trade secret of codeMantra, LLC and is protected by the copyright laws of the United States'}, page_content='597 - 598  \n  where to start,   599 - 600  \n  wireless keyboards,   611  \n  wireless mice,   611   \n   focus problems (digital cam-\neras),   529 - 531   \n   folders  \n  AppData,   252  \n  HomeGroup folders, opening, \n  247 - 249   \n   Format command,   452 - 454   \n   formatting  \n  drives,   452 - 454  \n  memory cards,   495 - 496   \n   

In [40]:
from langchain_openai import OpenAI
llm= OpenAI(temperature=0.4, max_tokens=500)

In [42]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt= (
    "You are an IT helpdesk assistant for a company." \
    "Use the following pieces of retrieved context to answer the question." \
    "If you do not know the answer, say 'I do not know the answer.'"
    "Keep your answer concise and easy to understand."
    "\n\n"
    "{context}"
)

prompt= ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [43]:
question_answer_chain= create_stuff_documents_chain(llm, prompt)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response= rag_chain.invoke({"input": "What is a computer?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}